For each decimal output, the first number is MSE, the second one is Pearson correlation coefficient, and the thrid value is corresponding p-value.

In [1]:
import os
import re
import nltk
import gensim
import readability
import numpy as np
import pandas as pd
import sklearn as sl
from nltk.parse import stanford
from scipy import spatial
from scipy.stats.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

In [2]:
os.environ['STANFORD_PARSER'] = "../Tools/stanford-parser-full-2018-10-17/stanford-parser.jar"
os.environ['STANFORD_MODELS'] = "../Tools/stanford-parser-full-2018-10-17/stanford-parser-3.9.2-models.jar"
parser = stanford.StanfordParser( model_path = "edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz" )

C:\Users\Ruosen\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: The StanfordParser will be deprecated
Please use nltk.parse.corenlp.CoreNLPParser instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
data_train = pd.read_excel( "Data/Train_Data.xlsx", sheet_name = "Sheet1" )
data_test  = pd.read_excel( "Data/Test_Data.xlsx", sheet_name = "Sheet1" )
stop_words = set( nltk.corpus.stopwords.words( "english" ) )

In [4]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format(
                "Data/GoogleNews-vectors-negative300.bin", binary = True )

In [5]:
# Basic data processing functions
"""Clean data

1. Replace tab or new line characters with space
2. Lowercase words
3. Remove extra spaces

Args:
    line: a string contains original sentence(s) or content(s).
    
Returns:
    line: a string contains cleaned sentence(s) or content(s).
"""
def clean( line ):
    line = re.sub( "[\s▃]", " ", line )
    line = re.sub( "_", "", line )
    line = re.sub( "[^\w\s]", "", line )
    line = re.sub( "\s+", " ", line ).strip()
    line.lower()
    return line

"""Preprocess all data in summary level

Args:
    data: a dataframe contains Summary text, Non-Redundancy score,
          and Fluency score.

Returns:
    new_summaries: a list of string which contains cleaned summary.
"""
def preprocess( data ):
    print( "Preprocessing..." )
    summaries = data["Summary"]
    new_summaries = []
    for summary in summaries:
        new_summaries.append( clean( summary ) )
    return new_summaries

"""Preprocess all data in sentence level

Args:
    data: a dataframe contains Summary text, Non-Redundancy score,
          and Fluency score.

Returns:
    new_summaries: a list of cleaned summary. Its structure is:
    
                   [[sentence1], [sentence2], ...]
"""
def preprocess_sentence( data ):
    # Split summaries into sentences and clean them
    summaries = data["Summary"]
    new_summaries = []
    for summary in summaries:
        sentences = summary.split( '.' )
        new_sentences = []
        for sentence in sentences:
            sentence = sentence.strip()
            new_sentence = clean( sentence )
            if len( new_sentence ) == 0:
                continue
            new_sentences.append( new_sentence )
        new_summaries.append( new_sentences )
    return new_summaries

"""Standardize data

Args:
    data: a numpy array.
    cache: a dictionary contains cached mean and standard diviation.
           Its structure is:
           
           {"mean": mean, "std": standard diviation}

Returns:
    std_data: a numpy data contains stardaized data.
    cache: a dictionary contains cached mean and standard diviation.
           Its structure is:
           
           {"mean": mean, "std": standard diviation}
"""
def standardize( data, cache = None ):
    if isinstance( data, list ):
        tdata = np.array( data )
    if cache == None:
        cache = {"mean": np.mean( tdata ),
                 "std" : np.std ( tdata )}
    std_data = ( tdata - cache["mean"] ) / cache["std"]
    if isinstance( data, list ):
        std_data = std_data.tolist()
    return std_data, cache

In [6]:
# Utils for problem 4.1
"""Get max unigram of each sentence

Maximum repetition of unigrams: calculate the frequencies of all unigrams
(remove stop words), and use the maximum value as the result.

Args:
    data: a dataframe contains Summary text, Non-Redundancy score,
          and Fluency score.
    stop_words: a set of stop words.
    cache: a dictionary contains cached mean and standard diviation.
           Its structure is:
           
           {"mean": mean, "std": standard diviation}

Returns:
    unigrams: a list of max unigram of each sentence corresponding to
              original sentences.
    cache: a dictionary contains cached mean and standard diviation.
           Its structure is:
           
           {"mean": mean, "std": standard diviation}
"""
def get_max_unigram( data, stop_words, cache = None ):
    print( "Geting maximum repetition of unigrams of each sentence..." )
    summaries = preprocess( data )
    unigrams = []
    for summary in summaries:
        unigram = {}
        words = summary.split()
        max_number = 0
        for word in words:
            if word in stop_words:
                continue
            if word not in unigram:
                unigram[word] = 0
            unigram[word] += 1
            max_number = max( max_number, unigram[word] )
        unigrams.append( max_number )
    unigrams, cache = standardize( unigrams, cache )
    return unigrams, cache

"""Get max bigram of each sentence

Maximum repetition of bigrams: calculate the frequencies of all bigrams,
and use the maximum value as the result.

Args:
    data: a dataframe contains Summary text, Non-Redundancy score,
          and Fluency score.
    cache: a dictionary contains cached mean and standard diviation.
           Its structure is:
           
           {"mean": mean, "std": standard diviation}

Returns:
    bigrams: a list of max bigram of each sentence corresponding to
            original sentences.
    cache: a dictionary contains cached mean and standard diviation.
           Its structure is:
           
           {"mean": mean, "std": standard diviation}
"""
def get_max_bigram( data, cache = None ):
    print( "Geting maximum repetition of bigrams of each sentence..." )
    summaries = preprocess( data )
    bigrams = []
    for summary in summaries:
        bigram = {}
        words = summary.split()
        max_number = 0
        prev_word = ""
        for i in range( len( words ) - 1 ):
            two_words = ' '.join( words[i: i + 2] )
            if two_words not in bigram:
                bigram[two_words] = 0
            bigram[two_words] += 1
            max_number = max( max_number, bigram[two_words] )
        bigrams.append( max_number )
    bigrams, cache = standardize( bigrams, cache )
    return bigrams, cache

"""Get max unigram of each sentence

Maximum sentence similarity: each sentence is represented as average of
word embeddings, then compute cosine similarity between pairwise sentences,
use the maximum similarity as the result.

Args:
    data: a dataframe contains Summary text, Non-Redundancy score,
          and Fluency score.
    word2vec: a word2vec model obtained from nltk.
    cache: a dictionary contains cached mean and standard diviation.
           Its structure is:
           
           {"mean": mean, "std": standard diviation}

Returns:
    sen_sim: a list of max sentence similarity of each sentence.
    cache: a dictionary contains cached mean and standard diviation.
           Its structure is:
           
           {"mean": mean, "std": standard diviation}
"""
def get_max_sentence_similarity( data, word2vec, cache = None ):
    print( "Geting maximum sentence similarity of each sentence..." )
    
    summaries = preprocess_sentence( data )
    
    # Calculate the average word embedding for sentences
    word2vec_dict = word2vec.vocab.keys()
    avgvecs_units = []
    for summary in summaries:
        avgvecs_sentences = []
        for sentence in summary:
            words = sentence.split()
            # 300 dims in word2vec for each word-0vector
            wordvecs = []
            for word in words:
                if word in word2vec_dict:
                    wordvecs.append( word2vec[word] )
            if len( wordvecs ) == 0:
                continue
            wordvecs = np.array( wordvecs )
            avgvecs_sentences.append( np.mean( wordvecs, axis = 0 ) )
        avgvecs_units.append( avgvecs_sentences )
    
    # Calculate max sentence similarity
    print( "Calculating cosine similarity..." )
    sen_sims = []
    for avgvecs in avgvecs_units:
        max_cos_sim = -100
        for i in range( len( avgvecs ) ):
            for j in range( len( avgvecs ) ):
                if i == j:
                    continue
                cos_sim = 1 - spatial.distance.cosine( avgvecs[i], avgvecs[j] )
                max_cos_sim = max( max_cos_sim, cos_sim )
        sen_sims.append( max_cos_sim )
    sen_sims, cache = standardize( sen_sims, cache )
    return sen_sims, cache

"""Get length for each sentence

Args:
    data: a dataframe contains Summary text, Non-Redundancy score,
          and Fluency score.
    cache: a dictionary contains cached mean and standard diviation.
           Its structure is:
           
           {"mean": mean, "std": standard diviation}

Returns:
    lengths: a list of length of each summary.
    cache: a dictionary contains cached mean and standard diviation.
           Its structure is:
           
           {"mean": mean, "std": standard diviation}
"""
def get_length( data, cache = None ):
    print( "Getting length of each sentence..." )
    summaries = preprocess( data )
    lengths = []
    for summary in summaries:
        lengths.append( len( summary.split() ) )
    lengths, cache = standardize( lengths, cache )
    return lengths, cache

"""Get ratio of stop words in each sentence

Calculate ratio of stop words in each sentences.

Args:
    data: a dataframe contains Summary text, Non-Redundancy score,
          and Fluency score.
    stop_words: a set of stop words.
    cache: a dictionary contains cached mean and standard diviation.
           Its structure is:
           
           {"mean": mean, "std": standard diviation}

Returns:
    ratio_of_stop_words: a list of ratio of stop words in each
                         sentence.
    cache: a dictionary contains cached mean and standard diviation.
           Its structure is:
           
           {"mean": mean, "std": standard diviation}
"""
def get_ratio_of_stop_words( data, stop_words, cache = None ):
    print( "Getting ratio of stop words of each sentence..." )
    summaries = preprocess( data )
    ratio_of_stop_words = []
    for summary in summaries:
        words = summary.split()
        cnt = 0
        for word in words:
            if word in stop_words:
                cnt += 1
        ratio_of_stop_words.append( cnt / len( word ) )
    ratio_of_stop_words, cache = standardize( ratio_of_stop_words, cache )
    return ratio_of_stop_words, cache

In [7]:
"""Train part of problem 4.1.1

Args:
    data: a dataframe contains Summary text, Non-Redundancy score,
          and Fluency score.
    stop_words: a set of stop words.
    word2vec: a word2vec model obtained from nltk.

Returns:
    model: a well-trained mlp model.
    caches: a dictionary contains cached mean and standard diviation
            of all features. Its structure is:
           
            {"unigram": {"mean": mean, "std": standard diviation},
             "bigram" : ...,
             "sen_sim": ...}
"""
def problem4_1_1_train( data, stop_words, word2vec ):
    print( "Problem 4.1.1 Train" )
    print( "Getting features..." )
    caches = {}
    max_unigram, caches["unigram"] = get_max_unigram( data, stop_words )
    max_bigram,  caches["bigram"]  = get_max_bigram( data )
    max_sen_sim, caches["sen_sim"] = get_max_sentence_similarity( data, word2vec )
    features = list( zip( max_unigram, max_bigram, max_sen_sim ) )
    labels = list( data["Non-Redundancy"] )
    print( "Building and training models..." )
    model = MLPRegressor( ( 5, ), activation = "tanh",
                          max_iter = 1000, learning_rate_init = 0.01 )
    model.fit( features, labels )
    return model, caches

"""Test part of problem 4.1.1

Args:
    data: a dataframe contains Summary text, Non-Redundancy score,
          and Fluency score.
    model: a trained model.
    stop_words: a set of stop words.
    word2vec: a word2vec model obtained from nltk.
    caches: a dictionary contains cached mean and standard diviation
            of all features. Its structure is:
           
            {"unigram": {"mean": mean, "std": standard diviation},
             "bigram" : ...,
             "sen_sim": ...}

Returns:
    None.
"""
def problem4_1_1_test( data, model, stop_words, word2vec, caches ):
    print( "Problem 4.1.1 Test" )
    print( "Getting features..." )
    max_unigram, _ = get_max_unigram( data, stop_words, caches["unigram"] )
    max_bigram,  _ = get_max_bigram( data, caches["bigram"] )
    max_sen_sim, _ = get_max_sentence_similarity( data, word2vec,
                                                  caches["sen_sim"] )
    features = list( zip( max_unigram, max_bigram, max_sen_sim ) )
    labels = list( data["Non-Redundancy"] )
    print( "Predicting..." )
    preds = model.predict( features )
    print( "Measuring..." )
    mses = mean_squared_error( labels, preds )
    pcor = pearsonr( labels, preds )
    print( mses, pcor )

In [8]:
model_4_1_1, caches = problem4_1_1_train( data_train, stop_words, word2vec )
problem4_1_1_test( data_test, model_4_1_1, stop_words, word2vec, caches )

Problem 4.1.1 Train
Getting features...
Geting maximum repetition of unigrams of each sentence...
Preprocessing...
Geting maximum repetition of bigrams of each sentence...
Preprocessing...
Geting maximum sentence similarity of each sentence...
Calculating cosine similarity...
Building and training models...
Problem 4.1.1 Test
Getting features...
Geting maximum repetition of unigrams of each sentence...
Preprocessing...
Geting maximum repetition of bigrams of each sentence...
Preprocessing...
Geting maximum sentence similarity of each sentence...
Calculating cosine similarity...
Predicting...
Measuring...
0.1779790558650628 (0.7447020756849336, 1.2806695318289553e-36)


In [9]:
"""Train part of problem 4.1.2

Args:
    data: a dataframe contains Summary text, Non-Redundancy score,
          and Fluency score.
    stop_words: a set of stop words.
    word2vec: a word2vec model obtained from nltk.
    func: a string represents which function it would use.

Returns:
    model: a well-trained MLP model.
    caches: a dictionary contains cached mean and standard diviation
            of all features. Its structure is:
           
            {"unigram": {"mean": mean, "std": standard diviation},
             "bigram" : ...,
             "sen_sim": ...,
             "length" : ...,
             "ratio"  : ...}
"""
def problem4_1_2_train( data, stop_words, word2vec, func ):
    print( "Problem 4.1.2 Train" )
    print( "Getting features..." )
    caches = {}
    max_unigram, caches["unigram"] = get_max_unigram( data, stop_words )
    max_bigram,  caches["bigram"]  = get_max_bigram( data )
    max_sen_sim, caches["sen_sim"] = get_max_sentence_similarity( data, word2vec )
    if func == "length":
        length, caches["length"] = get_length( data )
        features = list( zip( max_unigram, max_bigram, max_sen_sim, length ) )
    else:
        ratio,  caches["ratio"]  = get_ratio_of_stop_words( data, stop_words )
        features = list( zip( max_unigram, max_bigram, max_sen_sim, ratio ) )
    labels = list( data["Non-Redundancy"] )
    print( "Building and training models..." )
    model = MLPRegressor( ( 5, ), activation = "tanh",
                           max_iter = 1000, learning_rate_init = 0.01 )
    model.fit( features, labels )
    return model, caches

"""Test part of problem 4.1.2

Args:
    data: a dataframe contains Summary text, Non-Redundancy score,
          and Fluency score.
    model: a trained model.
    stop_words: a set of stop words.
    word2vec: a word2vec model obtained from nltk.
    func: a string represents which function it would use.
    caches: a dictionary contains cached mean and standard diviation
            of all features. Its structure is:
           
            {"unigram": {"mean": mean, "std": standard diviation},
             "bigram" : ...,
             "sen_sim": ...,
             "length" : ...,
             "ratio"  : ...}

Returns:
    None.
"""
def problem4_1_2_test( data, model, stop_words, word2vec, func, caches ):
    print( "Problem 4.1.2 Test" )
    print( "Getting features..." )
    max_unigram, _ = get_max_unigram( data, stop_words, caches["unigram"] )
    max_bigram,  _ = get_max_bigram( data, caches["bigram"] )
    max_sen_sim, _ = get_max_sentence_similarity( data, word2vec,
                                                  caches["sen_sim"] )
    if func == "length":
        length, _ = get_length( data, caches["length"] )
        features  = list( zip( max_unigram, max_bigram, max_sen_sim, length ) )
    else:
        ratio, _ = get_ratio_of_stop_words( data, stop_words, caches["ratio"] )
        features = list( zip( max_unigram, max_bigram, max_sen_sim, ratio ) )
    labels = list( data["Non-Redundancy"] )
    print( "Predicting by model..." )
    preds = model.predict( features )
    print( "Measuring..." )
    mses = mean_squared_error( labels, preds )
    pcor = pearsonr( labels, preds )
    print( mses, pcor )

In [10]:
func = "length"
print( "Try to add feature " + func )
model_4_1_2_1, caches = problem4_1_2_train( data_train, stop_words, word2vec, func )
problem4_1_2_test( data_test, model_4_1_2_1, stop_words, word2vec, func, caches )
func = "stop_words"
print( "Try to add feature " + func )
model_4_1_2_2, caches = problem4_1_2_train( data_train, stop_words, word2vec, func )
problem4_1_2_test( data_test, model_4_1_2_2, stop_words, word2vec, func, caches )

Try to add feature length
Problem 4.1.2 Train
Getting features...
Geting maximum repetition of unigrams of each sentence...
Preprocessing...
Geting maximum repetition of bigrams of each sentence...
Preprocessing...
Geting maximum sentence similarity of each sentence...
Calculating cosine similarity...
Getting length of each sentence...
Preprocessing...
Building and training models...
Problem 4.1.2 Test
Getting features...
Geting maximum repetition of unigrams of each sentence...
Preprocessing...
Geting maximum repetition of bigrams of each sentence...
Preprocessing...
Geting maximum sentence similarity of each sentence...
Calculating cosine similarity...
Getting length of each sentence...
Preprocessing...
Predicting by model...
Measuring...
0.17563927750208272 (0.7513204661151712, 1.371308915137041e-37)
Try to add feature stop_words
Problem 4.1.2 Train
Getting features...
Geting maximum repetition of unigrams of each sentence...
Preprocessing...
Geting maximum repetition of bigrams of 

In [11]:
# Utils for problem 4.2
"""Get number of repetitive unigram

Args:
    data: a dataframe contains Summary text, Non-Redundancy score,
          and Fluency score.
    cache: a dictionary contains cached mean and standard diviation.
           Its structure is:
           
           {"mean": mean, "std": standard diviation}

Returns:
    cnts: a list contains stardardized count value of all sentences.
    cache: a dictionary contains cached mean and standard diviation.
           Its structure is:
           
           {"mean": mean, "std": standard diviation}
"""
def get_no_of_repetitive_unigram( data, cache = None ):
    cnts = []
    summaries = preprocess( data )
    for summary in summaries:
        words = summary.split()
        unigram = {}
        cnt = 0
        for word in words:
            if word not in unigram:
                unigram[word] = 0
            if unigram[word]:
                cnt += 1
            unigram[word] += 1
        cnts.append( cnt )
    cnts, cache = standardize( cnts, cache )
    return cnts, cache

"""Get number of repetitive bigram

Args:
    data: a dataframe contains Summary text, Non-Redundancy score,
          and Fluency score.
    cache: a dictionary contains cached mean and standard diviation.
           Its structure is:
           
           {"mean": mean, "std": standard diviation}

Returns:
    cnts: a list contains stardardized count value of all sentences.
    cache: a dictionary contains cached mean and standard diviation.
           Its structure is:
           
           {"mean": mean, "std": standard diviation}
"""
def get_no_of_repetitive_bigram( data, cache = None ):
    cnts = []
    summaries = preprocess( data )
    for summary in summaries:
        words = summary.split()
        bigram = {}
        cnt = 0
        for i in range( len( words ) ):
            two_words = ' '.join( words[i:i + 2] )
            if two_words not in bigram:
                bigram[two_words] = 0
            if bigram[two_words]:
                cnt += 1
            bigram[two_words] += 1
        cnts.append( cnt )
    cnts, cache = standardize( cnts, cache )
    return cnts, cache

"""Get easy read score

Args:
    data: a dataframe contains Summary text, Non-Redundancy score,
          and Fluency score.
    cache: a dictionary contains cached mean and standard diviation.
           Its structure is:
           
           {"mean": mean, "std": standard diviation}

Returns:
    scores: a list contains stardardized scores of all sentences.
    cache: a dictionary contains cached mean and standard diviation.
           Its structure is:
           
           {"mean": mean, "std": standard diviation}
"""
def get_easy_read_score( data, cache = None ):
    scores = []
    summaries = preprocess_sentence( data )
    for sentences in summaries:
        min_score = 10000
        for sentence in sentences:
            sentence = sentence.strip()
            if len( sentence ) == 0:
                continue
            words = sentence.split()
            score = readability.getmeasures( words, lang = 'en' )
            score = score['readability grades']['FleschReadingEase']
            min_score = min( min_score, score )
        scores.append( min_score )
    scores, cache = standardize( scores, cache )
    return scores, cache

"""Get parser height of all sentences

Args:
    data: a dataframe contains Summary text, Non-Redundancy score,
          and Fluency score.
    parser: a stanford parser for parsing english PENN tree.
    cache: a dictionary contains cached mean and standard diviation.
           Its structure is:
           
           {"mean": mean, "std": standard diviation}

Returns:
    heights: a list contains stardardized heights of all sentences.
    cache: a dictionary contains cached mean and standard diviation.
           Its structure is:
           
           {"mean": mean, "std": standard diviation}
"""
def get_parser_height( data, parser, cache = None ):
    summaries = preprocess_sentence( data )
    heights = []
    cnt = 0
    for sentences in summaries:
        height = []
        for sentence in sentences:
            sentence = sentence.strip()
            if len( sentence ) == 0:
                continue
            parsed_sentence = parser.parse_sents( sentence )
            height.append( next( next( parsed_sentence ) ).height () )
            cnt += 1
            print( "Parsed sentences: " + str( cnt ), end = "\r" )
        heights.append( np.mean( height ) )
    print( "" )
    heights, cache = standardize( heights, cache )
    return heights, cache

"""Get phrase propotion of all sentences

For each summary, calculate the ratio of all pos-tags starts with "IN",
"NN", or "VB".

Args:
    data: a dataframe contains Summary text, Non-Redundancy score,
          and Fluency score.
    cache: a dictionary contains cached mean and standard diviation.
           Its structure is:
           
           {"mean": mean, "std": standard diviation}

Returns:
    phrase_ratio: a list contains stardardized phrase ratio of all
                  sentences.
    cache: a dictionary contains cached mean and standard diviation.
           Its structure is:
           
           {"mean": mean, "std": standard diviation}
"""
def get_phrase_propotion( data, cache = None ):
    summaries = preprocess_sentence( data )
    phrase_ratio = []
    total_cnt = 0
    for sentences in summaries:
        ratio = []
        cnt = 0
        cnt_tag = 0
        for sentence in sentences:
            sentence = sentence.strip()
            if len( sentence ) == 0:
                continue
            pos_tags = nltk.pos_tag( sentence )
            for pos_tag in pos_tags:
                cnt += 1
                if pos_tag[1].startswith( "IN" ) or \
                        pos_tag[1].startswith( "NN" ) or \
                        pos_tag[1].startswith( "VB" ):
                    cnt_tag += 1
            total_cnt += 1
            print( "Parsed sentence: " + str( total_cnt ), end = "\r" )
        phrase_ratio.append( cnt_tag / cnt )
    print( "" )
    phrase_ratio, cache = standardize( phrase_ratio, cache )
    return phrase_ratio, cache

In [12]:
"""Train part of problem 4.2.1

Args:
    data: a dataframe contains Summary text, Non-Redundancy score,
          and Fluency score.
    stop_words: a set of stop words.

Returns:
    model: a well-trained MLP model.
    caches: a dictionary contains cached mean and standard diviation
            of all features. Its structure is:
           
            {"rep_unigram": {"mean": mean, "std": standard diviation},
             "rep_bigram" : ...,
             "rd_score"   : ...}
"""
def problem4_2_1_train( data, stop_words ):
    print( "Problem 4.2.1 Train" )
    print( "Getting features..." )
    caches = {}
    rep_uni,  caches["rep_unigram"] = get_no_of_repetitive_unigram( data )
    rep_bi,   caches["rep_bigram"]  = get_no_of_repetitive_bigram( data )
    rd_score, caches["rd_score"]    = get_easy_read_score( data )
    features = list( zip( rep_uni, rep_bi, rd_score ) )
    labels = list( data["Fluency"] )
    print( "Building and training models..." )
    model = MLPRegressor( ( 5, ), activation = "tanh",
                          max_iter = 1000, learning_rate_init = 0.01 )
    model.fit( features, labels )
    return model, caches

"""Test part of problem 4.2.1

Args:
    data: a dataframe contains Summary text, Non-Redundancy score,
          and Fluency score.
    model: a trained model.
    stop_words: a set of stop words.
    caches: a dictionary contains cached mean and standard diviation
            of all features. Its structure is:
           
            {"rep_unigram": {"mean": mean, "std": standard diviation},
             "rep_bigram" : ...,
             "rd_score"   : ...}

Returns:
    None.
"""
def problem4_2_1_test( data, model, stop_words, caches ):
    print( "Problem 4.2.1 Test" )
    print( "Getting features..." )
    rep_uni,  _ = get_no_of_repetitive_unigram( data, caches["rep_unigram"] )
    rep_bi,   _ = get_no_of_repetitive_bigram ( data, caches["rep_bigram"] )
    rd_score, _ = get_easy_read_score( data, caches["rd_score"] )
    features = list( zip( rep_uni, rep_bi, rd_score ) )
    labels = list( data["Fluency"] )
    print( "Predicting..." )
    preds = model.predict( features )
    print( "Measuring..." )
    mses = mean_squared_error( labels, preds )
    pcor = pearsonr( labels, preds )
    print( mses, pcor )

In [13]:
model_4_2_1, caches = problem4_2_1_train( data_train, stop_words )
problem4_2_1_test( data_test, model_4_2_1, stop_words, caches )

Problem 4.2.1 Train
Getting features...
Preprocessing...
Preprocessing...
Building and training models...
Problem 4.2.1 Test
Getting features...
Preprocessing...
Preprocessing...
Predicting...
Measuring...
0.23833074210188449 (0.29134845986416164, 2.8437665383259195e-05)


In [14]:
"""Train part of problem 4.2.2

Args:
    data: a dataframe contains Summary text, Non-Redundancy score,
          and Fluency score.
    stop_words: a set of stop words.
    parser: a stanford parser for parsing english PENN tree.
    func: a string represents which function it would use.
    caches: a dictionary contains cached mean and standard diviation
            of all features. Its structure is:
           
            {"rep_unigram": {"mean": mean, "std": standard diviation},
             "rep_bigram" : ...,
             "rd_score"   : ...,
             "parser_height": ...,
             "phrase_prop"  : ...}

Returns:
    model: a well-trained MLP model.
"""
def problem4_2_2_train( data, stop_words, parser, func ):
    print( "Problem 4.2.2 Train" )
    print( "Getting features..." )
    caches = {}
    rep_unigram, caches["rep_unigram"] = get_no_of_repetitive_unigram( data )
    rep_bigram,  caches["rep_bigram"]  = get_no_of_repetitive_bigram ( data )
    rd_score,    caches["rd_score"]    = get_easy_read_score( data )
    if func == "height":
        parser_height, caches["parser_height"] = get_parser_height( data, parser )
        features = list( zip( rep_unigram, rep_bigram, rd_score, parser_height ) )
    else:
        phrase_prop, caches["phrase_prop"] = get_phrase_propotion( data )
        features = list( zip( rep_unigram, rep_bigram, rd_score, phrase_prop ) )
    labels = list( data["Non-Redundancy"] )
    print( "Building and training models..." )
    model = MLPRegressor( ( 5, ), activation = "tanh",
                           max_iter = 1000, learning_rate_init = 0.01 )
    model.fit( features, labels )
    return model, caches

"""Test part of problem 4.2.2

Args:
    data: a dataframe contains Summary text, Non-Redundancy score,
          and Fluency score.
    model: a trained model.
    stop_words: a set of stop words.
    parser: a stanford parser for parsing english PENN tree.
    func: a string represents which function it would use.
    caches: a dictionary contains cached mean and standard diviation
            of all features. Its structure is:
           
            {"rep_unigram": {"mean": mean, "std": standard diviation},
             "rep_bigram" : ...,
             "rd_score"   : ...,
             "parser_height": ...,
             "phrase_prop"  : ...}

Returns:
    None.
"""
def problem4_2_2_test( data, model, stop_words, parser, func, caches ):
    print( "Problem 4.2.2 Test" )
    print( "Getting features..." )
    rep_unigram, _ = get_no_of_repetitive_unigram( data, caches["rep_unigram"] )
    rep_bigram,  _ = get_no_of_repetitive_bigram ( data, caches["rep_bigram"] )
    rd_score,    _ = get_easy_read_score( data, caches["rd_score"] )
    if func == "height":
        parser_height, _ = get_parser_height( data, parser, caches["parser_height"] )
        features = list( zip( rep_unigram, rep_bigram, rd_score, parser_height ) )
    else:
        phrase_prop, _ = get_phrase_propotion( data, caches["phrase_prop"] )
        features = list( zip( rep_unigram, rep_bigram, rd_score, phrase_prop ) )
    labels = list( data["Non-Redundancy"] )
    print( "Predicting by model..." )
    preds = model.predict( features )
    print( "Measuring..." )
    mses = mean_squared_error( labels, preds )
    pcor = pearsonr( labels, preds )
    print( mses, pcor )

In [15]:
func = "height"
print( "Try to add feature " + func )
model_4_2_2_1, caches = problem4_2_2_train( data_train, stop_words, parser, func )
problem4_2_2_test( data_test, model_4_2_2_1, stop_words, parser, func, caches )
func = "phrase ratio"
print( "Try to add feature " + func )
model_4_2_2_2, caches = problem4_2_2_train( data_train, stop_words, parser, func )
problem4_2_2_test( data_test, model_4_2_2_2, stop_words, parser, func, caches )

Try to add feature height
Problem 4.2.2 Train
Getting features...
Preprocessing...
Preprocessing...
Parsed sentences: 2969
Building and training models...
Problem 4.2.2 Test
Getting features...
Preprocessing...
Preprocessing...
Parsed sentences: 753
Predicting by model...
Measuring...
0.20144800022429155 (0.7026233600234025, 4.387885956980907e-31)
Try to add feature phrase ratio
Problem 4.2.2 Train
Getting features...
Preprocessing...
Preprocessing...
Parsed sentence: 2969
Building and training models...
Problem 4.2.2 Test
Getting features...
Preprocessing...
Preprocessing...
Parsed sentence: 753
Predicting by model...
Measuring...
0.2007871260613363 (0.7042513146699327, 2.7947393979652166e-31)
